In [13]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "philadelphia_pennsylvania.osm"  # Replace this with your osm file
SAMPLE_FILE = "sample.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

In [56]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import pprint
import re
import codecs
import json

OSM_FILE = "philadelphia_pennsylvania.osm" 


In [57]:
def count_tags(filename):
    # YOUR CODE HERE
    dict_ = {}
    for event,elem in ET.iterparse(filename):
        if elem.tag not in dict_:
            dict_[elem.tag] = 1
        else:
            dict_[elem.tag] += 1
    return dict_


tags = count_tags(OSM_FILE)
#tags = count_tags(SAMPLE_FILE)
pprint.pprint(tags)

{'bounds': 1,
 'member': 61840,
 'nd': 4058993,
 'node': 3347875,
 'osm': 1,
 'relation': 5350,
 'tag': 2097223,
 'way': 349965}


In [58]:
# Regular expression for each category
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

lo = set()
lo_co = set()
pro_co = set()
oth = set()

def key_type(element, keys):
    if element.tag == "tag":
        # YOUR CODE HERE
        low = lower.search(element.attrib['k'])
        low_col = lower_colon.search(element.attrib['k'])
        prob = problemchars.search(element.attrib['k'])
        if low:
            keys["lower"] += 1
            lo.add(element.attrib['k']) 
        elif low_col:
            keys["lower_colon"] +=1
            lo_co.add(element.attrib['k']) 
        elif prob:
            keys["problemchars"] +=1
            pro_co.add(element.attrib['k']) 
        else:
            keys["other"] +=1
            oth.add(element.attrib['k'])
        
    return keys

def write_data(data, filename):
    with open(filename, 'wb') as f:
        for x in data:
            f.write(x + "\n")

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}

    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)
    
    write_data(lo, 'lower.txt')
    write_data(lo_co, 'lower_colon.txt')
    write_data(pro_co, 'problem_chars.txt')
    write_data(oth, 'other.txt')
    return keys


keys = process_map(OSM_FILE)
#keys = process_map(SAMPLE_FILE)
pprint.pprint(keys)

{'lower': 1003842, 'lower_colon': 909734, 'other': 183638, 'problemchars': 9}


In [59]:
lo = set()
lo_co = set()


def read_data(filename):
    with open(filename, 'rb') as f:
        lines = f.readlines()
        for l in lines:
            spl = l.strip('\n').split(":")
            if len(spl) == 2:
                #extract the parent key
                lo_co.add(spl[0])
            else:
                lo.add(spl[0])
                
            

read_data('lower.txt')
read_data('lower_colon.txt')
common = lo.intersection(lo_co)
pprint.pprint(common)

set(['abandoned',
     'access',
     'addr',
     'aerialway',
     'alt_name',
     'area',
     'bicycle',
     'bridge',
     'building',
     'bus',
     'butcher',
     'capacity',
     'change',
     'colour',
     'community',
     'construction',
     'crossing',
     'cycleway',
     'description',
     'destination',
     'diet',
     'disused',
     'healthcare',
     'heritage',
     'hgv',
     'historic',
     'internet_access',
     'is_in',
     'junction',
     'lanes',
     'maxheight',
     'maxspeed',
     'memorial',
     'motor_vehicle',
     'motorcycle',
     'name',
     'note',
     'official_name',
     'old_name',
     'oneway',
     'opening_hours',
     'operator',
     'passing_places',
     'payment',
     'phone',
     'placement',
     'public_transport',
     'railway',
     'ramp',
     'ref',
     'restriction',
     'route',
     'ruins',
     'service',
     'short_name',
     'site_status',
     'social_facility',
     'source',
     'source_ref

In [60]:
#Regular expression to find the street type
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]


def audit_street_type(street_types, street_name):
     # find street type using regex
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        
        #if street type not in expected list, we add the new street type and add the name along with it
        if street_type not in expected:
            street_types[street_type].add(street_name)


#returns true if key tag is a street name
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types


#st_types = audit(SAMPLE_FILE)
st_types = audit(OSM_FILE)

pprint.pprint(dict(st_types))

{'1': set(['Bloomfield Dr, Unit 1',
           'Route 1',
           'S Newtown Street Rd #1',
           'Sansom St #1',
           'Walnut St #1']),
 '118': set(['Upland Ave #118']),
 '168': set(['Marlton Pike East Ste. 168']),
 '17': set(['Lancaster Avenue #17']),
 '19047': set(['200 Manor Ave. Langhorne, PA 19047',
               '2245 E. Lincoln Hwy, Langhorne, PA 19047',
               '2275 E Lincoln Hwy, Langhorne, PA 19047',
               '2300  East Lincoln Highway, Pennsylvania 19047']),
 '19067': set(['East Trenton Avenue Morrisville, PA 19067']),
 '2': set(['Buck Rd #2']),
 '205': set(['Office Center Dr #205']),
 '206': set(['Route 206', 'US 206', 'US 70 & US 206']),
 '3': set(['Main St #3']),
 '302': set(['Route 73 North, Suite 302']),
 '315': set(['Heritage Center Dr #315']),
 '33': set(['33', 'Route 33']),
 '37th': set(['N 37th']),
 '38': set(['New Jersey 38', 'Route 38', 'State Route 38']),
 '39th': set(['N 39th']),
 '40': set(['1140 US Highway 40', 'Rt 40']),
 '4080'

In [63]:
mapping = {
        'Ave':  'Avenue',
        "Ave.": "Avenue",
        "Avene": "Avenue",
        'Blvd.': 'Boulevard',
        "Blvd": "Boulevard",
        "Boulevard,": "Boulevard",
        "Blvd,": "Boulevard",
        "Blvd.,": "Boulevard",
        "Dr": "Drive",
        "Dr.": "Drive",
        "Ct": "Court",
        "Ct.": "Court",
        "court": "Court",
        "Cv": "Cove",
        "cove": "Cove",
        "Cc": "Cove",
        'E':'East',
        'E.':'East',
        'N.':'North',
        "N": "North",
        "Pl": "Place",
        "Pl.": "Place",
        'Pkwy':'Parkway',
        "lane": "Lane",
        "Ln": "Lane",
        'Rd':'Road',
        "Rd.": "Road",
        "Hwy": "Highway",
        "HWY": "Highway",
        "Hwy,": "Highway",
        "H": "Highway",
        'S':'South',
        'S.':'South',
        'St':'Street',
        "St.": "Street",
        "st":"Street",
        "street":"Street",
        "Street,":"Street",
        'W':'West'
        }


all_street_word = set()
name_list = []

def audit_street(osmfile):
    osm_file = open(osmfile, "r")
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag in["node", "way", "relation"] :
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    better_name = update_name(tag.attrib['v'])
                    name_list.append(better_name)
                    
    osm_file.close()
    return name_list

def update_name(name):

    # YOUR CODE HERE
    for n in name.split():
        if n in mapping:
            name = name.replace(n, mapping[n])
    return name



st_name = audit_street(SAMPLE_FILE)
pprint.pprint(st_name)

['Martin Luther King Jr Boulevard',
 'Makefield Road',
 'Middle Holland Road',
 'Wilmington Pike',
 'Princeton Pike',
 'Cooper Street',
 'Quarry Street',
 'Windsor Avenue',
 'Brandywine Avenue',
 'Germantown Avenue',
 'Germantown Avenue',
 'Germantown Avenue',
 'Race Street',
 'Dilworthtown Road',
 'South Broad Street',
 'North Broad Street',
 'North Broad Street',
 'Stenton Avenue',
 'West Lancaster Avenue',
 'Lancaster Avenue',
 'Swedesford Road',
 'Warren Road',
 'Huntington Road',
 'Mulhlenburg Drive',
 'Thomas Road',
 'Thomas Road',
 'Von Steuben Drive',
 'Thomas Road',
 'Thomas Road',
 'Prussian Lane',
 'Prussian Lane',
 'Prussian Lane',
 'West Valley Road',
 'West Valley Road',
 'West Valley Road',
 'Walker Road',
 'Walker Road',
 'Walker Road',
 'Baron Dekalb Road',
 'General Scott Road',
 'General Scott Road',
 'Colonel Dewees Road',
 'Colonel Dewees Road',
 'Anthony Wayne Drive',
 'Brookmead Road',
 'Red Coat Lane',
 'Red Coat Lane',
 'Red Coat Lane',
 'Sullivan Drive',
 'Sul

 'Byron Terrace',
 'Chanticleer',
 'Pendleton Drive',
 'Kingsdale Avenue',
 'The Woods',
 'The Woods',
 'Chanticleer',
 'Chanticleer',
 'East Evesham Road',
 'Palmwood Avenue',
 'The Woods',
 'Heron Road',
 'The Woods',
 'The Woods',
 'Downing Street',
 'Berlin Road',
 'Sleepy Hollow Place',
 'Castle Drive',
 'The Woods',
 'Stonehenge Road',
 '1st Avenue',
 'Manor House Drive',
 'Willowdale Drive',
 'Queen Ann Road',
 'Chanticleer',
 'Junewood Drive',
 'Lamp Post Lane',
 'Mews Lane',
 'Manning Lane',
 'Cardinal Lake Drive',
 'State Route 70',
 'West Split Rock Drive',
 'Laurel Hill Drive',
 'The Woods Ii',
 'Downing Street',
 'Downing Street',
 'Lamp Post Lane',
 'Mews Lane',
 'Cuffys Lane',
 'Inskeep Court',
 'Kipling Road',
 'East Evesham Road',
 'Summer Place',
 'Brian Drive',
 'Swallow Drive',
 'Lakeside Avenue',
 'Rue Du Bois',
 'The Woods Ii',
 'Downing Street',
 'Chanticleer',
 'Squire Lane',
 'Ticonderoga Lane',
 'Cardinal Lake Drive',
 'Cobblestone Lane',
 'Chanticleer',
 'Eas

 'Lincoln Lane',
 'Halifax Court',
 'Brandywine Drive',
 'Stanton Lane',
 'Marbel Avenue',
 'Jonathan Lane',
 'Devon Road',
 'Hearthstone Lane',
 'North Maple Avenue',
 'Brandywine Drive',
 'South Maple Avenue',
 'Old Marlton Pike East',
 'Delancey Way',
 'Georgia Okeefe Way',
 'Peter Hurd Way',
 'Buxmont Road',
 'Summit Court',
 'Craine Drive',
 'Cambridge Avenue',
 'Baltimore Drive',
 'Sharps Run Court',
 'Alexandra Court',
 'Derby Court',
 'Abbotsford Drive',
 'Woodthrush Court',
 'Candlewood Circle',
 'Crown Prince Drive',
 'Mallard Drive',
 'Clydesdale Court',
 'Columbia Drive',
 'Benchly Way',
 'Hawk Lane',
 'Lowell Drive',
 'Hibiscus Drive',
 'Prince William Court',
 'Masters Circle',
 'Valley Forge Court',
 'Peter Hurd Way',
 'State Route 73',
 'Delancey Way',
 'Georgia Okeefe Way',
 'Grand Banks Circle',
 'Yorktown Drive',
 'John Singer Sargent Way',
 'Waterview Court',
 'Pickwick Drive',
 'Colony Trail',
 'Alexandra Court',
 'Hawk Lane',
 'Preamble Drive',
 'Olympia Drive',
 

In [64]:
k = set()
for event, element in ET.iterparse(SAMPLE_FILE):
    if element.tag == "tag" or element.tag == "way":
        for tag in element.iter("tag"):
             k.add(tag.attrib['k'])

In [65]:
len(k)

604

In [66]:
k

{'1',
 'Comment',
 'FIXME',
 'FIXME2',
 'FIXME3',
 'FIXME:hgv',
 'GPS_Date',
 'GPS_Time',
 'HFCS',
 'Horz_Prec',
 'MATERIAL',
 'Max_PDOP',
 'NAME',
 'NHD:ComID',
 'NHD:Elevation',
 'NHD:FCode',
 'NHD:FDate',
 'NHD:FTYPE',
 'NHD:FType',
 'NHD:GNIS_ID',
 'NHD:GNIS_Name',
 'NHD:Permanent_',
 'NHD:RESOLUTION',
 'NHD:ReachCode',
 'NHD:Resolution',
 'NHD:way_id',
 'NHS',
 'NJDOT_SRI',
 'NOTE',
 'PA:ANALGROUP',
 'PA:DIVISION',
 'PARK_NAME',
 'Phone',
 'SHAPE_LENG',
 'Std_Dev',
 'Suite',
 'TODO',
 'TRL_LENGTH',
 'Vert_Prec',
 'WIDTH',
 'Website',
 'abandoned:railway',
 'abbr',
 'abutters',
 'access',
 'acreage',
 'addr:block',
 'addr:city',
 'addr:country',
 'addr:full',
 'addr:housename',
 'addr:housenumber',
 'addr:housenumber_1',
 'addr:housenumber_2',
 'addr:housenumber_3',
 'addr:housenumber_4',
 'addr:interpolation',
 'addr:postcode',
 'addr:state',
 'addr:street',
 'addr:street:source',
 'addr:suite',
 'addr:unit',
 'admin_level',
 'aerialway',
 'aeroway',
 'alt_name',
 'amenity',
 'are

In [67]:
def is_phone(element):
    return (element.attrib['k'] == "phone" or element.attrib['k'] == "contact:phone")

In [68]:
phone_re = re.compile(r'^\d\d\d\-\d\d\d\-\d\d\d\d$')

In [69]:
for event, element in ET.iterparse(SAMPLE_FILE):
    if element.tag == "node" or element.tag == "way":
        for tag in element.iter("tag"):
            if is_phone(tag):
                if phone_re.search(tag.attrib['v']) == None:
                    print tag.attrib['v']

(610) 558-8000
+16109333410
+1 610 6446759
+16106886246
+1 856 624 4173
6102512227
(609) 894-8222
+1 215 5421269
+1 610 933 7471
(215) 222-2234
+1 484 6815402
(856) 243-2191
2157320125
+1 609 7710274
267.908.4694
+1 215 441 9600
+1 215 218 0808
+1 267 687 8258
+1 215 339 5177
+1 267 639 4506
(215) 438-6300
(215) 848-5883
(215) 924-2628
(215) 848-5658, (215) 848-9660
(215) 849-6614
(215) 939-3594
215 386 2989
215 724 8303
215 334 2620
215 336 8111
215 735 1050
267 250 1825
215 424 4800
215 225 5560
215 200 8256
215 684 8285
+1 215 809 1742
(215) 644-9074
(856) 256-4389
+1-610-617-0993
610 539 6700
(856) 665-4824
(856) 813-3100
(215) 885-1496
2159785469
(609) 394-5502
+1-609-695-5155
(609) 894-9080
+1 215 7319900
1-856-795-1373
+1-856-354-0054
+1-215-699-0650
(215) 343-1600
+1 856-429-4911
+1(215)891-5501
+1 484 924 9205
+1 610-917-8888
(215) 463-4123
+1 215-392-9030
(856) 384-8750
+1 856 757 3500
(215) 898-7341
+1 215 829 8200
+1 609 561 1700
+1 215 683 9813
(302-421-1753
+1-610-644-404

In [70]:
def update_phone(number):
    phone_re = re.compile(r'^\d\d\d\-\d\d\d\-\d\d\d\d$')
    if phone_re.search(number) == None:
        phno = []
        number = list(number.lstrip("+1"))
        for char in number:
            try:
                if int(char) in [x for x in range(10)]:
                    phno.append(char)
            except ValueError:
                continue
        number = "".join(phno)
        number = number[:3] + "-" + number[3:6] + "-" + number[6:]
                    
    return number

In [71]:
update_phone("+1 215-575-4000")

'215-575-4000'

In [72]:
for event, element in ET.iterparse(SAMPLE_FILE):
    for tag in element.iter("tag"):
            if is_phone(tag):
                print tag.attrib['v'], "=>", update_phone(tag.attrib['v'])

(610) 558-8000 => 610-558-8000
(610) 558-8000 => 610-558-8000
+16109333410 => 610-933-3410
+16109333410 => 610-933-3410
610-649-3061 => 610-649-3061
610-649-3061 => 610-649-3061
+1 610 6446759 => 610-644-6759
+1 610 6446759 => 610-644-6759
+16106886246 => 610-688-6246
+16106886246 => 610-688-6246
+1 856 624 4173 => 856-624-4173
+1 856 624 4173 => 856-624-4173
6102512227 => 610-251-2227
6102512227 => 610-251-2227
(609) 894-8222 => 609-894-8222
(609) 894-8222 => 609-894-8222
215-629-0500 => 215-629-0500
215-629-0500 => 215-629-0500
+1 215 5421269 => 215-542-1269
+1 215 5421269 => 215-542-1269
+1 610 933 7471 => 610-933-7471
+1 610 933 7471 => 610-933-7471
(215) 222-2234 => 215-222-2234
(215) 222-2234 => 215-222-2234
+1 484 6815402 => 484-681-5402
+1 484 6815402 => 484-681-5402
(856) 243-2191 => 856-243-2191
(856) 243-2191 => 856-243-2191
215-879-1454 => 215-879-1454
215-879-1454 => 215-879-1454
215-493-6160 => 215-493-6160
215-493-6160 => 215-493-6160
2157320125 => 215-732-0125
215732012

(302) 429-7600 => 302-429-7600
(302) 429-7600 => 302-429-7600
+1 856-334-5091 => 856-334-5091
+1 856-334-5091 => 856-334-5091
+1 609-380-7100 => 609-380-7100
+1 609-380-7100 => 609-380-7100
+1-215-572-1622 => 215-572-1622
+1-215-572-1622 => 215-572-1622
(610) 558-8000 => 610-558-8000
+16109333410 => 610-933-3410
610-649-3061 => 610-649-3061
+1 610 6446759 => 610-644-6759
+16106886246 => 610-688-6246
+1 856 624 4173 => 856-624-4173
6102512227 => 610-251-2227
(609) 894-8222 => 609-894-8222
215-629-0500 => 215-629-0500
+1 215 5421269 => 215-542-1269
+1 610 933 7471 => 610-933-7471
(215) 222-2234 => 215-222-2234
+1 484 6815402 => 484-681-5402
(856) 243-2191 => 856-243-2191
215-879-1454 => 215-879-1454
215-493-6160 => 215-493-6160
2157320125 => 215-732-0125
215-922-3745 => 215-922-3745
215-629-0999 => 215-629-0999
+1 609 7710274 => 609-771-0274
215-339-0900 => 215-339-0900
267-688-6493 => 267-688-6493
267.908.4694 => 267-908-4694
610-658-0618 => 610-658-0618
+1 215 441 9600 => 215-441-9600


In [73]:
def update_phone(number):
    phone_re = re.compile(r'^\d\d\d\-\d\d\d\-\d\d\d\d$')
    if phone_re.search(number) == None:
        phno = []
        number = list(number.lstrip("+1"))
        for char in number:
            try:
                if int(char) in [x for x in range(10)]:
                    if len(phno) == 10:
                        continue
                    phno.append(char)
            except ValueError:
                continue
        number = "".join(phno)
        number = number[:3] + "-" + number[3:6] + "-" + number[6:]
                    
    return number

In [74]:
def is_postcode(element):
    return (element.attrib['k'] == "addr:postcode" or element.attrib['k'] == "postal_code")

In [78]:
postcode_re = re.compile(r'^1\d\d\d\d$')

In [79]:
counter = 0
for element in get_element(OSM_FILE):
    if counter == 25:
        break
    if element.tag == "node" or element.tag == "way":
        for tag in element.iter("tag"):
            if is_postcode(tag):
                if postcode_re.search(tag.attrib['v']) == None:
                    print tag.attrib['v']
                    counter += 1

08611
PA
08096
19030-4005
19087-3696
08062
08034
08109
08096
08080
08062
08062
08080
08030
19096-9998
08098
08098
19010-3224
08098
PA
08071
08071
08071
08071
08071


In [80]:
def update_postcode(postcode):
    try:
        postcode = re.compile(r'7\d\d\d\d').search(postcode).group()
    except AttributeError:
        postcode = 'None'
    return postcode

In [81]:
for element in get_element(OSM_FILE):
    if element.tag == "node" or element.tag == "way":
        for tag in element.iter("tag"):
            if is_postcode(tag):
                if postcode_re.search(tag.attrib['v']) == None:
                    print tag.attrib['v'], "=>", update_postcode(tag.attrib['v'])

08611 => None
PA => None
08096 => None
19030-4005 => None
19087-3696 => None
08062 => None
08034 => None
08109 => None
08096 => None
08080 => None
08062 => None
08062 => None
08080 => None
08030 => None
19096-9998 => None
08098 => None
08098 => None
19010-3224 => None
08098 => None
PA => None
08071 => None
08071 => None
08071 => None
08071 => None
08071 => None
08071 => None
08071 => None
08071 => None
08071 => None
08071 => None
08071 => None
08071 => None
08071 => None
08071 => None
08071 => None
08071 => None
08071 => None
08071 => None
08071 => None
08071 => None
08071 => None
08071 => None
08071 => None
08098 => None
08035 => None
08031 => None
08045 => None
08035 => None
08034 => None
08033-2001 => None
08054 => None
19041-1228 => None
08505 => None
08086 => None
08096 => None
08068 => None
08068 => None
08068 => None
08096 => None
08096 => None
08096 => None
08096 => None
08096 => None
08505 => None
08021 => None
08021 => None
08109 => None
08096 => None
08096 => None
08096 => N

08003-3300 => None
08034 => None
08034 => None
19803-9997 => None
08091 => None
08109 => None
08034 => None
08034 => None
08035 => None
08028 => None
08107 => None
08062 => None
08062 => None
08062 => None
08062 => None
08094 => None
08012 => None
08108 => None
08610 => None
08628 => None
08628 => None
08030 => None
08010 => None
08103 => None
08048 => None
08034 => None
08037 => None
08048 => None
08086 => None
08619 => None
08085 => None
08071 => None
08071 => None
08071 => None
08105 => None
08102 => None
08102 => None
7500 => None
08094 => None
08033 => None
08080 => None
08053 => None
08009 => None
08026 => None
08505 => None
08043 => None
08043 => None
08043 => None
08505 => None
08619 => None
08619 => None
08619 => None
08002 => None
08009 => None
08003 => None
DE 19720 => None
08026 => None
08043 => None
08062 => None
08053 => None
08055 => None
08534 => None
08060 => None
08003 => None
08091 => None
08106 => None
08106 => None
08106 => None
08043-1028 => None
08043-1050 => Non

08043-4808 => None
08043-4773 => None
08043-9552 => None
08043-2041 => None
08043-3729 => None
08043 => None
08043-4604 => None
08043-3679 => None
08043-2058 => None
08043-4963 => None
08043-3405 => None
08043-4205 => None
08043-9535 => None
08043-3734 => None
08043-1140 => None
08043-3683 => None
08043-1417 => None
08043-1678 => None
08043-2500 => None
08043-3638 => None
08043-3401 => None
08043-4844 => None
08043-4743 => None
08043-2905 => None
08043-2843 => None
08043-2056 => None
08043-3016 => None
08043 => None
08043-1403 => None
08043-3415 => None
08043-2906 => None
08043-2545 => None
08043-3677 => None
08043-3413 => None
08043-2058 => None
08043-4845 => None
08043-4702 => None
08043-1114 => None
08043-2512 => None
08043-3624 => None
08043-2002 => None
08043-3401 => None
08043-4728 => None
08043-2461 => None
08043-4955 => None
08043-2232 => None
08043-1536 => None
08043-2051 => None
08043-1148 => None
08043-2929 => None
08043-1835 => None
08043-1820 => None
08043-4811 => None
080

08043 => None
08043-1629 => None
08043-3011 => None
08043-3011 => None
08043-3428 => None
08043-1203 => None
08043-3912 => None
08043-3919 => None
08043-3920 => None
08043-2821 => None
08043-1335 => None
08043-4711 => None
08043-3677 => None
08043-4811 => None
08043-1513 => None
08043-1867 => None
08043-2494 => None
08043-4745 => None
08043-9593 => None
08043-1819 => None
08043-3677 => None
08043-2843 => None
08043-2005 => None
08043-1136 => None
08043-1172 => None
08043-4938 => None
08043-2024 => None
08043-2902 => None
08043-2801 => None
08043-1171 => None
08043-3312 => None
08043-2840 => None
08043-2065 => None
08043-4860 => None
08043-3635 => None
08043-1609 => None
08043-1685 => None
08043-4756 => None
08043-1205 => None
08043-3012 => None
08043-3002 => None
08043-2849 => None
08043-2005 => None
08043-2942 => None
08043-2048 => None
08043-1861 => None
08043 => None
08043-2935 => None
08043-1216 => None
08043-1233 => None
08043-2868 => None
08043-2819 => None
08043-2060 => None
080

08043-2069 => None
08043-3202 => None
08043-3201 => None
08043-9562 => None
08043-1231 => None
08043-3682 => None
08043-1032 => None
08043-2971 => None
08043-4814 => None
08043-1605 => None
08043-2265 => None
08043-1035 => None
08043-1178 => None
08043-2355 => None
08043-4851 => None
08043-1817 => None
08043-1261 => None
08043 => None
08043-1258 => None
08043-3949 => None
08043-1837 => None
08043-4842 => None
08043-2351 => None
08043-3731 => None
08043 => None
08043-4724 => None
08043-1432 => None
08043-2008 => None
08043-1540 => None
08043-4698 => None
08043-4850 => None
08043-2050 => None
08043-4850 => None
08043-1651 => None
08043-1140 => None
08043-1829 => None
08043-4701 => None
08043-4310 => None
08043-1688 => None
08043-3682 => None
08043-3720 => None
08043-1671 => None
08043-2273 => None
08043-3428 => None
08043-1160 => None
08043-1511 => None
08043-4616 => None
08043-3809 => None
08043-2851 => None
08043-2303 => None
08043-3009 => None
08043-2972 => None
08043-4726 => None
080

...

08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 

08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 

08053 => None
08053 => None
08053 => None
08053 => None
08053 => None
08053 => None
08053 => None
08053 => None
08053 => None
08053 => None
08053 => None
08053 => None
08053 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08003-3367 => None
08003-3367 => None
08003-3367 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08068 => None
08043-1698 => None
08043-1698 => None
08043-1698 => None
08043-1698 => None
08043-1698 => None
08043-1698 => None
08043-1698 => None
08043-1698 => None
08043-1698 => None
08043-1698 => None
08043-1698 => None
08043-1698 => None
08043-1698 => None
08043-1698 => None
08091 => None
08091 => None
08091 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043

08034 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08003 => None
08003-2215 => None
08003-2725 => None
08003-2760 => None
08003-2756 => None
08003-2608 => None
08003-2756 => None
08003-2756 => None
08053 => None
08053 => None
08053-3007 => None
08053-3000 => None
08053 => None
08053 => None
08053 => None
08053 => None
08053 => None
08003 => None
08003 => None
08003 => None
08003 => None
08003 => None
08003 => None
08003 => None
08003 => None
08003 => None
08003 => None
19072b1393e => None
08043 => None
08033 => None
08033 => None
08033 => None
08033 => None
08033 => None
08033 => None
08033 => None
08033 => None
08033 => None
08033 => None
08033 => None
08033 => None
08033 => None
08033 => None
08043 => None
08003 => None
08053 => None
08043 => None
08033 => None
08053-2073 => None
08053-2073 => None
08053-2073 => None
08043 => None
08648 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => N

08043 => None
08043 => None
08043 => None
08055 => None
08055 => None
08055 => None
08055 => None
08055 => None
08055 => None
08690 => None
08850 => None
08026 => None
08057 => None
08057 => None
08054 => None
08054 => None
08054 => None
08054 => None
08054 => None
08054 => None
08054 => None
08054 => None
08054 => None
08054 => None
08054 => None
08054 => None
08054 => None
08054 => None
08084 => None
08084 => None
PA 19132 => None
08053 => None
08053 => None
08053 => None
08053 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043-3441 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043 => None
08043-3447 => None
08043 =

In [82]:
def is_city(element):
    return element.attrib['k'] == "addr:city"

In [91]:
expectedcities = ['Abington', 'Ambler', 'Ardmore', 'Aston', 'Aston Mills', 'Aston Township', 'Atco', 'Audubon',
                  'Audubon','Berlin','Abbington','Aldan','Andalusia','Audubon Park', 'Bala Cynwyd','Bellmawr',
                'Belmont Hills','Bensalem','Bethayres','Bethel Township','Beverly','Blackwood', 'Boothwyn', 'Bordentown City', 
'Berwyn', 'Blandon' ,'Blue Bell' ,'Bricktown' ,' Brookmall', ' Burlington',
                  'Bridgeport','Bristol', 'Brookhaven', 'Brooklawn', 'Broomall','Bryn Mawr','Buckingham',"Carney's Point",
             'Castle','Chalfont','Cheltenham','Camden', 'Chadds Ford', 'Cherry Hill', 
'Chester', 'Columbus', 'Chesilhurst','Chesterbrook','Cinnaminson Township', 'Claymont', 'Collegeville','Collingswood',
         'Colmar', 'Concord Township', 'Conshohocken', 'Deptford', 'Devon', 'Doylestown','Dresher', 'Drexel Hill','Eagleville',
    'East Haddonfield', 'East Norriton', 'Edgewater Park', 'Elkins Park', 'Erdenheim', 'Evesham', 'Ewing',
 'Crosswicks','Douglassville', 'Downingtown', 'Dublin', 'Exton', 'Fairless Hills','Feasterville-Trevose', 'Flemington',  'Fort Washington', 
'Franconia', 'Glassboro', 'Gloucester City', 'Glen Mills', 'Glenolden','Gwynedd',
 
 
 'Glenside','Hamilton', 'Holland', 'Lindenwold', 'Linwood', 'Logan Township', 'Lawrenceville', 
 'Lower Merion Township', 'Lumberton','Malvern', 'Mantua', 'Maple Glen', 'Maple Shade','Marcus Hook',
 'Marlton', 'Medford', 'Media','Merchantville','Merion Station','Monroeville', 'Montgomeryville', 'Moorestown',
                  'Morton', 'Mount Holly', 'Mount Laurel', 'Mullica Hill'
                  'Narberth', 'Netown', 'New Britain', 'New Castle', 'Newtown',
                  'Norristown', 'North Hills', 'Norwood', 'Oaklyn', 'Oaks', 'Ogden', 'Oreland',
                  'Palmyra', 'Paoli','Paulsboro', 'Pemberton', 'Penn Valley', 'Penndel', 'Pennsauken',
               'Kulpsville','King Of Prussia', 'Lafayette Hill', 'Langhorne', 'Lansdale', 'Lansdowne', 'Lawnside',
 'Lawrence Township', 'Lawrenceville', 'Lenni', 'Limerick',  'Levittown', 'Marlboro' ,'Morrisville', 'North Wales', 
'Palinsboro', 'Pennington','Philadelphia', 'Plainsboro','Phoenixville', 'Pilesgrove','Pineville', 'Pitman','Plymouth Meeting',
 'Pottstown', 'Prospect Park', 'Radnor', 'Richboro', 'Riverton', 'Rockledge','Rosemont',
 'Roslyn', 'Royersford', 'Rushland', 'Schwenksville', 'Rosemont','Sewell',
 'Sharon Hill', 'Sicklerville', 'Skippack', 'Somerdale', 'Souderton', 'Spring House', 'Springfield', 'Stratford',
                  'Sewell', 'Sharon Hill', 'Swarthmore', 'Swedesboro', 'Telford', 'Trenton', 
                  'Trevose', 'Turnersville', 'Upland', 'Upper Chichester', 'Upper Darby',
                  'Villanova', 'Vincentown', 'Voorhees', 'Warminster', 'Warrington',
                  'Wayne', 'West Berlin', 'West Chester', 'West Deptford', 'West Windsor',
                  'Westmont', 'Williamstown', 'Willingboro', 'Willow Grove', 'Wilmington', 'Woodbury', 'Woodbury Heights',
                   'Southampton', 'Spring City','Wyncote', 'Wynnewood','Yardley', 'Temple', 'Zieglerville ']

In [90]:
cities = set()
for element in get_element(OSM_FILE):
    if element.tag == "node" or element.tag == "way":
        for tag in element.iter("tag"):
            if is_city(tag):
                if tag.attrib['v'] not in expectedcities:
                    cities.add(tag.attrib['v'])

In [88]:
cities

{'Boothwyn;Upper Chichester',
 'Bordentown Township',
 'Cheltenham Township',
 'Cherry Hill Township',
 'Cherry hill',
 'Doylestown Township',
 'Ewing Township',
 'Ewing twp',
 'Fairfax, DE',
 'Feasterville-Trevose',
 'Folsom',
 'Forest Grove',
 'Frazer',
 'Furlong',
 'Garnet Valley',
 'Gibbsboro',
 'Gladwyne',
 'Glen Mills',
 'Glen Mills, Pennsylvania',
 'Glen Mills, Pennsyvania',
 'Glen Mills. Pennsylvania',
 'Glenn Mills',
 'Glenolden',
 'Glenside',
 'Gwynedd',
 'Haddon Heights',
 'Haddon Township',
 'Haddonfield',
 'Haddontownship',
 'Hamilton Square',
 'Hamilton Township',
 'Hammonton',
 'Harleysville',
 'Hatboro',
 'Haverford',
 'Haverford Township',
 'Havertown',
 'Hopewell twp',
 'Horsham',
 'Horsham Township',
 'Huntingdon Valley',
 'Ivyland',
 'Jamison',
 'Jenkintown',
 'King Of Prussia',
 'King of Prussia',
 'Lawrence Twp',
 'Lawrence twp',
 'Limerick Township',
 'Maple Shade Township',
 'Medford Township',
 'Moorestown Township',
 'Mount Holly Township',
 'Mount Laurel Town

In [92]:
mapping_city = {
    "": "Philadelphia",
    'Boothwyn;Upper Chichester' : 'Boothwyn',
 'Bordentown Township': 'Bordentown',
 'Cheltenham Township': 'Cheltenham',
 'Cherry Hill Township' : 'Cherry Hill',
 'Cherry hill' : 'Cherry Hill',
 'Doylestown Township' : 'Doylestown',
 'Ewing Township': 'Ewing',
 'Ewing twp': 'Ewing',
 'Fairfax, DE': 'Philadelphia',
 'Feasterville-Trevose': 'Feasterville Trevose',
 'Folsom': 'Folsom',
 'Forest Grove': 'ForestGrove',
 'Frazer': 'Frazer',
 'Furlong': 'Furlong',
 'Garnet Valley' : 'Garnet Valley',
 'Gibbsboro': 'Gibbsboro',
 'Gladwyne': 'Gladwyne',
 'Glen Mills': 'Glen Mills',
 'Glen Mills, Pennsylvania': 'Glen Mills',
 'Glen Mills, Pennsyvania': 'Glen Mills',
 'Glen Mills. Pennsylvania': 'Glen Mills',
 'Glenn Mills': 'Glen Mills',
 
 'Haddon Heights': 'Haddon Heights',
 'Haddon Township': 'Haddon Township',
 'Haddonfield': 'Haddonfield',
 'Haddontownship': 'Hadoon Township',
 'Hamilton Square': 'Hamilton Square',
 'Hamilton Township': 'Hamilton Township',
 'Hammonton': 'Hamilton',
 'Harleysville': 'Philadelphia',
 'Hatboro': 'Hatboro',
 'Haverford': 'Haverford',
 'Haverford Township': 'Haverford',
 'Havertown':'Havertown',
 'Hopewell twp': 'Hopewell Township',
 'Horsham': 'Horsham',
 'Horsham Township': 'Horsham',
 'Huntingdon Valley' : 'Huntingdon Valley',
 'Ivyland': 'Philadelphia',
 'Jamison': 'Jamison',
 'Jenkintown': 'Jenkintown',
 'King Of Prussia': 'King Of Prussia',
 'King of Prussia': 'King Of Prussia',
 'Lawrence Twp':'Lawrenceville',
 'Lawrence twp' : 'Lawrenceville',
 'Limerick Township' : 'Limerick',
 'Maple Shade Township': 'Maple Shade',
 'Medford Township': 'Medford',
 'Moorestown Township': 'Moorestown',
 'Mount Holly Township' : 'Mount Holly',
 'Mount Laurel Township' : 'Mount Laurel',
 'Mount Royal': 'Mount Laurel',
 'Mt. Laurel' : 'Mount Laurel',
 'Mullica HIll' : 'Mullica Hill',
 'Mullica Hill' : 'Mullica Hill',
 'Mullica Hll' : 'Mullica Hill',
 'Narberth' : 'Narberth',
 'Newtown Square' : 'Newtown Square',
 'Newtown Township': 'Newtown',
 'PA': 'Philadelphia',
 'Pennsauken Township' : 'Pennsauken',
 'Pennsville' : 'Phoenixville',
 'Phialdephia' : 'Philadelphia',
 'Philadelpha' : 'Philadelphia',
 'Philadelphia, PA' : 'Philadelphia',
 'Philadelphias' : 'Philadelphia',
 'Philaelphia' : 'Philadelphia',
 'Phildadelphia' : 'Philadelphia',
 'Springfield Township' : 'Springfield Township',
 'Strafford' : 'Stratford',
 'Trenton city' : 'Trenton',
 'Upper Darby Township' : 'Philadelphia',
 'Upper Dublin Township': 'Philadelphia',
 'Voorhees Township' : 'Voorhees',
 'Wallingford' : 'Philadelphia',
 'Warminster Township': 'Warminster',
 'Warrington Township' : 'Warrington',
 'Washington Township' : 'Washington',
 'West Norriton Township' :'Norriton' ,
 'West Trenton': ' Trenton',
 'West Windsor Township' : 'Philadelphia',
 'West windsor twp' : 'Philadelphia',
 'Willistown Township' : 'Willimstown',
 'Wilmington, DE' :' Wilmington',
 'Wilmington, DE 19806': 'Wilmington',
 'Wilminton' : 'Wilmington',
 'Woodstown' : 'Wrightstown',
 'Woolwich Township' :'Woolwich',
 'Wrightstown':'Wrightstown',
 'bensalem' : 'Bensalem',
 'jenkintown' :'Jenkintown',
 'lansdale' : 'Lansdale',
 'norristown' : 'Norristown',
 'phialdephia' : 'Philadelphia',
 'phila' : 'Philadelphia',
 'philadelphia' : 'Philadelphia',
 'trooper' :'Trappe',
 'woodbury ' : 'Woodbury'
}

In [93]:
def update_city(city):
    if city not in expectedcities:
        if city in mapping_city.keys():
            city = mapping_city[city]
        else:
            city = "None"
    return city

In [94]:
for element in get_element(OSM_FILE):
    if element.tag == "node" or element.tag == "way":
        for tag in element.iter("tag"):
            if is_city(tag):
                if tag.attrib['v'] not in expectedcities:
                    print tag.attrib['v'], "=>", update_city(tag.attrib['v'])

philadelphia => Philadelphia
Wallingford => Philadelphia
Newtown Square => Newtown Square
Huntingdon Valley => Huntingdon Valley
Horsham => Horsham
Jenkintown => Jenkintown
Mullica HIll => Mullica Hill
Trenton city => Trenton
Lawrence twp => Lawrenceville
Lawrence twp => Lawrenceville
Mullica HIll => Mullica Hill
Mullica Hill => Mullica Hill
Huntingdon Valley => Huntingdon Valley
King of Prussia => King Of Prussia
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Hatboro => Hatboro
Medford Township => Medford
Jenkintown => Jenkintown
King of Prussia => King Of Prussia
Woodstown => Wrightstown
Cherry Hill Township => Cherry Hill
Doylestown Township => Doylestown
Doylestown Township => Doylestown
Mullica Hill => Mullica Hill
Harleysville => Philadelphia
Jenkintown => Jenkintown
Hatboro => Hatboro
Philadelphia, PA => Philadelphia
Washington Township => Washington
Horsham => Horsham
King of Prussia => King Of Prussia
Woodstown => Wrightstown
Woodstown => W

Lawrence twp => Lawrenceville
Lawrence twp => Lawrenceville
Lawrence twp => Lawrenceville
Lawrence twp => Lawrenceville
Lawrence twp => Lawrenceville
Lawrence twp => Lawrenceville
West windsor twp => Philadelphia
West windsor twp => Philadelphia
West windsor twp => Philadelphia
West windsor twp => Philadelphia
West windsor twp => Philadelphia
West windsor twp => Philadelphia
West windsor twp => Philadelphia
West windsor twp => Philadelphia
West windsor twp => Philadelphia
Lawrence twp => Lawrenceville
Lawrence twp => Lawrenceville
Lawrence twp => Lawrenceville
Lawrence twp => Lawrenceville
Lawrence twp => Lawrenceville
Lawrence twp => Lawrenceville
Lawrence twp => Lawrenceville
Lawrence twp => Lawrenceville
Lawrence twp => Lawrenceville
Lawrence twp => Lawrenceville
Lawrence twp => Lawrenceville
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
T

Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => 

Ewing twp => Ewing
Ewing twp => Ewing
Ewing twp => Ewing
Hopewell twp => Hopewell Township
Hopewell twp => Hopewell Township
Hopewell twp => Hopewell Township
Hopewell twp => Hopewell Township
Hopewell twp => Hopewell Township
Hopewell twp => Hopewell Township
Hopewell twp => Hopewell Township
Hopewell twp => Hopewell Township
Hopewell twp => Hopewell Township
Hopewell twp => Hopewell Township
Hopewell twp => Hopewell Township
Hopewell twp => Hopewell Township
Hopewell twp => Hopewell Township
Hopewell twp => Hopewell Township
Hopewell twp => Hopewell Township
Hopewell twp => Hopewell Township
Hopewell twp => Hopewell Township
Hopewell twp => Hopewell Township
Hopewell twp => Hopewell Township
Hopewell twp => Hopewell Township
Hopewell twp => Hopewell Township
Lawrence twp => Lawrenceville
Lawrence twp => Lawrenceville
Lawrence twp => Lawrenceville
Lawrence twp => Lawrenceville
Lawrence twp => Lawrenceville
Lawrence twp => Lawrenceville
Lawrence twp => Lawrenceville
Lawrence twp => Law

Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => Trenton
Trenton city => 

...

Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => N

Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => N

Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => Narberth
Narberth => N

Gladwyne => Gladwyne
Gladwyne => Gladwyne
Maple Shade Township => Maple Shade
lansdale => Lansdale
Forest Grove => ForestGrove
Forest Grove => ForestGrove
Forest Grove => ForestGrove
Forest Grove => ForestGrove
Wilmington, DE 19806 => Wilmington
Wilmington, DE 19806 => Wilmington
Maple Shade Township => Maple Shade
Jenkintown => Jenkintown
Jenkintown => Jenkintown
Jenkintown => Jenkintown
Jenkintown => Jenkintown
Jenkintown => Jenkintown
Jenkintown => Jenkintown
Jenkintown => Jenkintown
Haddonfield => Haddonfield
Haddonfield => Haddonfield
Haddonfield => Haddonfield
Haddonfield => Haddonfield
Haddonfield => Haddonfield
Haddonfield => Haddonfield
Haddonfield => Haddonfield
Haddonfield => Haddonfield
Haddonfield => Haddonfield
Haddonfield => Haddonfield
Haddonfield => Haddonfield
Haddonfield => Haddonfield
Haddonfield => Haddonfield
Haddonfield => Haddonfield
Haddonfield => Haddonfield
Haddonfield => Haddonfield
Haddonfield => Haddonfield
Haddonfield => Haddonfield
Haddonfield => Haddonf

Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => Furlong
Furlong => F

In [95]:
def update_city(city, expectedcities, mapping_city):
    if city not in expectedcities:
        if city in mapping_city.keys():
            city = mapping_city[city]
        else:
            city = "None"
    return city

In [96]:
update_city("Philadelphia, PA", expectedcities, mapping_city)

'Philadelphia'

In [97]:
import xml.etree.cElementTree as ET
import csv
import codecs
import cerberus
import schema
import re

ImportError: No module named cerberus